# 1) Titolo e obiettivi

Lezione 32: Sentiment Analysis - Classificare la polarità del testo

---

## Mappa della lezione

| Sezione | Contenuto | Tempo stimato |
|---------|-----------|---------------|
| 1 | Titolo, obiettivi, cos'è sentiment analysis | 5 min |
| 2 | Teoria: rappresentazioni, modelli, metriche | 15 min |
| 3 | Schema mentale: pipeline sentiment | 5 min |
| 4 | Demo: BoW + NB, TF-IDF + LR, interpretazione | 25 min |
| 5 | Esercizi guidati + error analysis | 15 min |
| 6 | Conclusione operativa | 10 min |
| 7 | Checklist di fine lezione + glossario | 5 min |
| 8 | Changelog didattico | 2 min |

---

## Obiettivi della lezione

Al termine di questa lezione sarai in grado di:

| # | Obiettivo | Verifica |
|---|-----------|----------|
| 1 | Costruire **pipeline sentiment** completa | Sai fare preprocessing → vectorize → classify? |
| 2 | Confrontare **BoW vs TF-IDF** per sentiment | Sai quale performa meglio? |
| 3 | Usare **NaiveBayes e LogisticRegression** | Sai quando preferire quale? |
| 4 | Valutare con **metriche appropriate** | Sai usare F1 su classi sbilanciate? |
| 5 | **Interpretare il modello** | Sai estrarre parole positive/negative? |

---

## L'idea centrale: cos'è Sentiment Analysis

```
INPUT (recensione):                  OUTPUT (sentiment):

"This product is amazing!           → POSITIVE (0.95)
 Best purchase ever!"

"Terrible quality, waste            → NEGATIVE (0.92)
 of money. Very disappointed."

"It's okay, nothing special."       → NEUTRAL/MIXED (0.55)
```

**Applicazioni:** recensioni prodotti, social media, feedback clienti, brand monitoring.

---

## Il problema delle negazioni

```
SENZA BIGRAMMI:                     CON BIGRAMMI:

"not good"                          "not good"
   ↓                                    ↓
token: ["not", "good"]              token: ["not", "good", "not good"]
   ↓                                    ↓
"good" → positivo!                  "not good" → negativo!
   ↓                                    ↓
ERRORE!                             CORRETTO!
```

**Soluzione:** usa `ngram_range=(1, 2)` per catturare negazioni.

---

## Modelli per Sentiment Analysis

| Modello | Pro | Contro | Quando usare |
|---------|-----|--------|--------------|
| **MultinomialNB** | Veloce, baseline | Assume indipendenza | Prototipo rapido |
| **LogisticRegression** | Interpretabile, robusto | Lento su grandi dataset | Produzione |
| **SVM (LinearSVC)** | Ottimo su sparse | Meno probabilistico | Alta precisione |
| **Deep Learning** | Cattura semantica | Richiede molti dati | Dataset grandi |

---

## Pipeline Sentiment Analysis

```
┌──────────────┐    ┌──────────────┐    ┌──────────────┐    ┌──────────────┐
│    Testo     │ →  │   Pulizia    │ →  │  Vectorizer  │ →  │  Classifier  │
│   grezzo     │    │  lowercase   │    │  TF-IDF      │    │  LogReg      │
└──────────────┘    │  punct       │    └──────────────┘    └──────────────┘
                    │  stopwords   │           │                    │
                    └──────────────┘           ▼                    ▼
                                        fit su TRAIN          predict su TEST
                                        transform su          evaluate F1
                                        TRAIN + TEST
```

---

## Prerequisiti

| Concetto | Dove lo trovi | Verifica |
|----------|---------------|----------|
| BoW | Lezione 30 | Sai usare CountVectorizer? |
| TF-IDF | Lezione 31 | Sai usare TfidfVectorizer? |
| Classificazione binaria | Lezioni 5-6 | Sai cosa sono precision/recall? |
| Metriche F1 | Lezione 17 | Sai interpretare classification_report? |

**Cosa useremo:** CountVectorizer, TfidfVectorizer, MultinomialNB, LogisticRegression, accuracy_score, classification_report.

# 2) Teoria concettuale
- Sentiment analysis: assegnare etichette positive/negative a testi.
- Rappresentazioni: BoW (conteggi) vs TF-IDF (pesi informativi).
- Modelli comuni: Naive Bayes (adatto a conteggi), Logistic Regression (lineare su feature sparse).
- Metriche: accuracy, precision, recall, F1; attenzione a dataset sbilanciati.


## Scelte di preprocessing
- Lowercase, rimozione punteggiatura/numeri, eventualmente stopword.
- Uso di `fit` sul train e `transform` su test; non rifittare il vocabolario.
- ngram_range per catturare bigrammi utili a negazioni.


# 3) Schema mentale / mappa decisionale
1. Pulizia base del testo.
2. Split train/test stratificato.
3. Vettorizzazione (BoW o TF-IDF) con vocabolario fit sul train.
4. Addestramento modello (NB o LR) e valutazione su test.
5. Confronto rappresentazioni e modelli; scelta in base a F1/accuracy.
6. Interpretazione: parole piu' pesanti/coef per capire il modello.


# 4) Sezione dimostrativa
- Demo 1: dataset di recensioni, BoW + MultinomialNB.
- Demo 2: TF-IDF + LogisticRegression e confronto metriche.
- Demo 3: parole piu' indicative per classe.


## Demo 1 - BoW + Naive Bayes
Perche': baseline rapida su conteggi. Checkpoint: vocabolario fit su train, shape coerenti, accuracy > 0.5.


## Demo 2 - TF-IDF + Logistic Regression
Perche': pesi informativi e modello lineare; confronto con baseline. Checkpoint: F1 test, nessun rifit su test.


## Demo 3 - Parole piu' importanti
Perche': interpretare il modello TF-IDF con i coefficienti della logistic regression.


In [ ]:
# Setup e dataset
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
import re
import warnings
warnings.filterwarnings('ignore')
np.random.seed(42)

recensioni = [
    ("Prodotto eccezionale, lo consiglio a tutti", 1),
    ("Ottimo acquisto, qualita eccellente", 1),
    ("Pessimo, arrivato rotto", 0),
    ("Non funziona, soldi sprecati", 0),
    ("Prezzo onesto e buona qualita", 1),
    ("Esperienza terribile, non comprare", 0),
    ("Prodotto difettoso e assistenza assente", 0),
    ("Spedizione veloce e prodotto perfetto", 1),
    ("Deluso, qualita bassa", 0),
    ("Fantastico, supera le aspettative", 1)
]
texts, labels = zip(*recensioni)
X_train_txt, X_test_txt, y_train, y_test = train_test_split(texts, labels, test_size=0.3, random_state=42, stratify=labels)
print(f"Train: {len(X_train_txt)}, Test: {len(X_test_txt)}")


In [ ]:
# Demo 1: BoW + MultinomialNB
bow_vec = CountVectorizer()
X_train_bow = bow_vec.fit_transform(X_train_txt)
X_test_bow = bow_vec.transform(X_test_txt)
assert X_train_bow.shape[0] == len(X_train_txt)

nb = MultinomialNB()
nb.fit(X_train_bow, y_train)
preds_bow = nb.predict(X_test_bow)
acc_bow = accuracy_score(y_test, preds_bow)
print(f"Accuracy BoW+NB: {acc_bow:.3f}")
print(classification_report(y_test, preds_bow, digits=3))


In [ ]:
# Demo 2: TF-IDF + Logistic Regression
vec_tfidf = TfidfVectorizer(ngram_range=(1,2), min_df=1)
X_train_tfidf = vec_tfidf.fit_transform(X_train_txt)
X_test_tfidf = vec_tfidf.transform(X_test_txt)
assert X_test_tfidf.shape[1] == X_train_tfidf.shape[1]

lr = LogisticRegression(max_iter=500, random_state=42)
lr.fit(X_train_tfidf, y_train)
preds_tfidf = lr.predict(X_test_tfidf)
acc_tfidf = accuracy_score(y_test, preds_tfidf)
print(f"Accuracy TF-IDF+LR: {acc_tfidf:.3f}")
print(classification_report(y_test, preds_tfidf, digits=3))


In [ ]:
# Demo 3: parole piu' indicative (TF-IDF + LR)
feature_names = vec_tfidf.get_feature_names_out()
coefs = lr.coef_[0]
idx_sorted = np.argsort(coefs)
neg_terms = [(feature_names[i], coefs[i]) for i in idx_sorted[:5]]
pos_terms = [(feature_names[i], coefs[i]) for i in idx_sorted[-5:][::-1]]
print("Top parole negative:", neg_terms)
print("Top parole positive:", pos_terms)


In [ ]:
# Placeholder: nessuna esecuzione necessaria


In [ ]:
# Placeholder: nessuna esecuzione necessaria


In [ ]:
# Placeholder: nessuna esecuzione necessaria


# 5) Esercizi svolti (passo-passo)
## Esercizio 32.1 - Pipeline completa
Obiettivo: implementare la pipeline (BoW/TF-IDF) e confrontare NB vs LR su un nuovo set di frasi.


In [ ]:
# Soluzione esercizio 32.1
frasi = [
    ("servizio clienti pessimo e tempi lunghi", 0),
    ("molto soddisfatto dell acquisto", 1),
    ("qualita scarsa e prodotto rotto", 0),
    ("spedizione rapida e prodotto conforme", 1),
    ("non vale il prezzo pagato", 0),
    ("ottimo rapporto qualita prezzo", 1)
]
texts2, labels2 = zip(*frasi)
Xtr, Xte, ytr, yte = train_test_split(texts2, labels2, test_size=0.33, random_state=42, stratify=labels2)
vec2 = TfidfVectorizer(ngram_range=(1,2))
Xtr_t = vec2.fit_transform(Xtr)
Xte_t = vec2.transform(Xte)
nb2 = MultinomialNB()
nb2.fit(Xtr_t, ytr)
preds = nb2.predict(Xte_t)
print(classification_report(yte, preds, digits=3))
assert preds.shape[0] == len(yte)


## Esercizio 32.2 - Error analysis
Obiettivo: identificare false positive/negative e verificare termini che generano errori.


In [ ]:
# Soluzione esercizio 32.2
misclassified = []
for text, true, pred in zip(X_test_txt, y_test, preds_tfidf):
    if true != pred:
        misclassified.append((text, true, pred))
print("Errori trovati:")
for e in misclassified:
    print(e)


## Esercizio 32.3 - Stopword e n-grammi
Obiettivo: misurare l'impatto di stopword rimosse e bigrammi sul punteggio.


In [ ]:
# Soluzione esercizio 32.3
stopwords_it = {'il','lo','la','i','gli','le','un','una','di','a','da','in','con','su','per','tra','fra','e','o','ma','che','non','sono','ho','ha','questo','questa','questi','queste'}
vec_sw = TfidfVectorizer(stop_words=stopwords_it, ngram_range=(1,2))
Xtr_sw = vec_sw.fit_transform(X_train_txt)
Xte_sw = vec_sw.transform(X_test_txt)
acc_sw = accuracy_score(y_test, lr.fit(Xtr_sw, y_train).predict(Xte_sw))
print(f"Accuracy con stopword e bigrammi: {acc_sw:.3f}")


# 6) Conclusione operativa

## 5 take-home messages

| # | Messaggio | Perché importante |
|---|-----------|-------------------|
| 1 | **Bigrammi per negazioni** | "not good" ≠ "good" |
| 2 | **TF-IDF spesso batte BoW** | Pesi informativi migliorano F1 |
| 3 | **LogReg interpretabile** | coef_ mostra parole positive/negative |
| 4 | **Stratify nello split** | Bilancia classi nel train/test |
| 5 | **Error analysis obbligatoria** | Guarda i falsi positivi/negativi |

---

## Confronto sintetico: BoW vs TF-IDF per sentiment

| Aspetto | BoW + NB | TF-IDF + LR |
|---------|----------|-------------|
| **Velocità training** | Molto veloce | Veloce |
| **F1 tipico** | 0.75-0.85 | 0.80-0.90 |
| **Interpretabilità** | Media | Alta (coef_) |
| **Gestione negazioni** | Richiede bigrammi | Richiede bigrammi |
| **Quando preferire** | Baseline, prototipo | Produzione |

---

## Perché questi metodi funzionano

### 1) MultinomialNB per testo

```
P(positivo | "great", "product") ∝ P("great" | positivo) × P("product" | positivo) × P(positivo)

Naive = assume che "great" e "product" siano indipendenti
→ Semplificazione, ma funziona sorprendentemente bene su testo!
```

### 2) LogisticRegression per interpretazione

```
score = w₁×"great" + w₂×"terrible" + w₃×"not_good" + ...

w₁ = +2.5  (parola positiva)
w₂ = -3.0  (parola negativa)
w₃ = -1.8  (bigramma negativo)

→ Ordina per peso = keyword positive/negative!
```

---

## Reference card metodi

| Metodo | Input | Output | Note |
|--------|-------|--------|------|
| `MultinomialNB()` | X sparse | predict, predict_proba | Per conteggi BoW |
| `LogisticRegression()` | X sparse | predict, predict_proba, coef_ | Interpretabile |
| `classification_report()` | y_true, y_pred | precision, recall, F1 | Metriche complete |
| `TfidfVectorizer(ngram_range=(1,2))` | docs | X sparse | Bigrammi inclusi |

---

## Errori comuni e debug rapido

| Errore | Perché sbagliato | Fix |
|--------|-----------------|-----|
| No bigrammi | Perde negazioni | ngram_range=(1,2) |
| Refit vectorizer su test | Leakage | Solo transform() |
| Ignora sbilanciamento | F1 ingannevole | stratify=y, macro F1 |
| Non guarda errori | Non capisci il modello | Analizza FP e FN |
| Troppi max_features | Overfitting | Prova 3000-5000 |

---

## Template Sentiment Analysis completo

```python
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import pandas as pd
import numpy as np

# 1) Prepara dati
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.2, stratify=labels, random_state=42
)

# 2) Vettorizza con bigrammi
vec = TfidfVectorizer(ngram_range=(1, 2), max_features=5000, min_df=2)
X_train_vec = vec.fit_transform(X_train)
X_test_vec = vec.transform(X_test)

# 3) Classifica
clf = LogisticRegression(max_iter=1000, random_state=42)
clf.fit(X_train_vec, y_train)
preds = clf.predict(X_test_vec)

# 4) Valuta
print(classification_report(y_test, preds))

# 5) Interpreta
feature_names = vec.get_feature_names_out()
coefs = clf.coef_[0]
top_pos = np.argsort(coefs)[-10:][::-1]
top_neg = np.argsort(coefs)[:10]
print("Parole POSITIVE:", [feature_names[i] for i in top_pos])
print("Parole NEGATIVE:", [feature_names[i] for i in top_neg])

# 6) Error analysis
errors = X_test[(preds != y_test)]
print(f"Errori: {len(errors)} / {len(y_test)}")
```

---

## Prossimi passi

| Lezione | Argomento | Collegamento |
|---------|-----------|--------------|
| 33 | Named Entity Recognition | Estrarre entità dal testo |
| 34 | Document Intelligence | Struttura documenti |
| 35+ | Information Retrieval | Ricerca semantica |

# 7) Checklist di fine lezione
- [ ] Ho pulito i testi e gestito stopword/negazioni.
- [ ] Ho fatto split stratificato e fit del vocabolario solo sul train.
- [ ] Ho confrontato BoW e TF-IDF e almeno due modelli (NB, LR).
- [ ] Ho controllato metriche su test (accuracy/F1) e analizzato errori.
- [ ] Ho interpretato le parole piu' pesanti del modello.

Glossario
- Bag of Words: rappresentazione a conteggi.
- TF-IDF: pesi basati su frequenza e rarita'.
- Stopword: termini molto frequenti e poco informativi.
- Bigramma: sequenza di 2 parole consecutive.
- Logistic Regression: classificatore lineare per probabilita'.
- MultinomialNB: Naive Bayes per conteggi.


# 8) Changelog didattico

| Versione | Data | Modifiche |
|----------|------|-----------|
| 1.0 | 2024-02-08 | Creazione: BoW + NB baseline |
| 1.1 | 2024-02-15 | Aggiunto TF-IDF + LR |
| 2.0 | 2024-02-20 | Integrata interpretazione coef_ |
| 2.1 | 2024-02-25 | Refactor con error analysis |
| **2.3** | **2024-12-19** | **ESPANSIONE COMPLETA:** mappa lezione 8 sezioni, tabella obiettivi, ASCII pipeline sentiment, problema negazioni con bigrammi, confronto modelli table, 5 take-home messages, NB vs LR explanation, template completo con interpretazione, error analysis guidata |

---

## Note per lo studente

Sentiment Analysis è uno dei task NLP più comuni:

| Variante | Classi | Esempio |
|----------|--------|---------|
| Binario | pos/neg | Recensioni prodotti |
| Ternario | pos/neu/neg | Social media |
| Fine-grained | 1-5 stelle | Rating |
| Aspect-based | Sentiment per aspetto | "Camera ottima, batteria pessima" |

**Pipeline standard:**
1. Pulizia → 2. Vettorizzazione → 3. Classificazione → 4. Interpretazione

**Prossima tappa:** Lesson 33 - Named Entity Recognition (NER)